Imports

In [78]:
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os

In [79]:
def secondsToUTC (dataset):
    #1989-12-31 00:00:00
    zero = datetime(1989,12,31,0,0,0)
    #print (datetime.strftime(zero,"%b %d %Y %H:%M:%S"))
    new_dataset = []
    for d in dataset:
        n = str(zero + timedelta(seconds=d))
        new_dataset.append(n)
    return new_dataset

In [80]:
def prepareCSV (filepath, file, filetype):
    #Read csv
    df = pd.read_csv(filepath+file+filetype)
    
    #filter for local number == 7 and type == Date
    df_filtered = df.loc[df['Local Number'] == 7]
    df_filtered = df_filtered.loc[df_filtered['Type'] == 'Data']
    
    #convert position and time
    df_filtered['Value 2'] = df_filtered['Value 2']* 180/(2**31)
    df_filtered['Units 2'] = "degrees"

    df_filtered['Value 3'] = df_filtered['Value 3']* 180/(2**31)
    df_filtered['Units 3'] = "degrees"

    df_filtered['Value 1'] = secondsToUTC(df_filtered['Value 1'])
    df_filtered['Units 1'] = "UTC"
    
    # drop unnecessary data
    toBeDropped = []
    counter = 10
    field_str = 'Field '
    value_str = 'Value '
    units_str = 'Units '

    while counter < 24:
        toBeDropped.append(field_str+str(counter))
        toBeDropped.append(value_str+str(counter))
        toBeDropped.append(units_str+str(counter))
        counter = counter + 1

    toBeDropped.append("Type")
    toBeDropped.append("Local Number")
    toBeDropped.append("Message")
    toBeDropped.append("Unnamed: 72")
    
    df_filtered = df_filtered.drop(columns=toBeDropped)
    
    # write csv
    df_filtered.to_csv(filepath+file+'_prepared'+filetype)

Get file names in directory


In [87]:
filepath = '/home/julio/strides/data/fit2csv/'
filetype = '.csv'
files = []
for file in os.listdir(filepath):
    if file.endswith(".csv") and not file.endswith("prepared.csv"):
        files.append(file.split('.')[0])

['913F2746', '91SI5602', '916A2805', '911E0950', '911D3633', '91VI5920', '914H2817', '921B4648', '91U70725', '91VI2925', '915F0120', '911B3000', '919H2158', '911C4903']


Prepare files

In [88]:
for file in files:
    prepareCSV (filepath, file, filetype)

save